In [2]:
%%capture
import pandas as pd
import numpy as np

import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaMulticore
from gensim.models.phrases import Phraser, Phrases

import spacy
spacy.load('en', disable=['parser', 'ner'])

!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words("english")
stop_words.extend(['say', 'would', 'also', 'use'])

from pprint import pprint

from google.colab import drive

In [3]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [4]:
# Load data
data = pd.read_csv(r'/content/gdrive/My Drive/Datasets/Text - News Dataset/articles1.csv')

In [5]:
data.head(3)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."


In [6]:
# Sample dataset to reduce time it takes to build model
data = data.sample(n=4000)
data.shape

(4000, 10)

In [8]:
def preprocess_text(texts, allowed_pos_tags=['NOUN', 'VERB', 'ADV', 'ADJ']):
  preprocessed_texts = list()
  # Use simple preprocess to convert to lower case and remove punctuation then add to list if NOT in list of stop words
  texts = [[token for token in simple_preprocess(doc, deacc=True) if token not in stop_words] for doc in texts]
  bigrams = Phrases(texts, min_count=2)
  trigrams = Phrases(bigrams[texts], min_count=2)
  bigram_model = Phraser(bigrams)
  trigram_model = Phraser(trigrams)
  bi_gram_texts = [bigram_model[doc] for doc in texts]
  tri_gram_texts = [trigram_model[doc] for doc in bi_gram_texts]

  nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'] )
  for doc in tri_gram_texts:
    doc = nlp(' '.join(doc))
    # Use spacy to lemmatize and conform token's pos tag is in the allowed list
    preprocessed_texts.append([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
  return preprocessed_texts

In [9]:
# Cleaned up text
texts = preprocess_text(data.content)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [10]:
# Create dictionary
id2word = corpora.Dictionary(texts)

# Create corpus
corpus = [id2word.doc2bow(text) for text in texts]

**Build LDA Model**

In [144]:
%%capture
# Get the coherence score for different number of topics
history = dict()
for i in range(20, 31):
  # Build LDA model with i number of topcics
  lda_model = LdaMulticore(corpus=corpus, num_topics=i, id2word=id2word, random_state=1, passes=5)
  # Get topic coherence and add to history
  coherence_model = gensim.models.CoherenceModel(model=lda_model, texts=texts, corpus=corpus, dictionary=id2word)
  history[i] = coherence_model.get_coherence()

In [145]:
sorted(history.items(), key=lambda x: x[1], reverse=True)

[(27, 0.2696518121703623),
 (21, 0.26705845864612376),
 (29, 0.2663829739764625),
 (30, 0.2641949471828947),
 (20, 0.2634629738242359),
 (24, 0.26310530729395426),
 (26, 0.2624653069413419),
 (25, 0.260377415167788),
 (22, 0.2602195490230664),
 (23, 0.2593544715970842),
 (28, 0.2550881806008998)]

In [11]:
%%capture
# Build LDA model with number of topics that had the highest coherence score
lda_model = LdaMulticore(corpus=corpus, num_topics=27, id2word=id2word, random_state=1, passes=5)

In [12]:
pprint(lda_model.print_topics())

[(22,
  '0.009*"say" + 0.006*"virus" + 0.005*"report" + 0.005*"man" + '
  '0.004*"session" + 0.004*"case" + 0.004*"show" + 0.003*"official" + '
  '0.003*"country" + 0.003*"address"'),
 (25,
  '0.010*"say" + 0.005*"report" + 0.005*"call" + 0.004*"government" + '
  '0.004*"campus" + 0.004*"may" + 0.004*"student" + 0.003*"country" + '
  '0.003*"claim" + 0.003*"man"'),
 (24,
  '0.009*"abortion" + 0.006*"say" + 0.006*"auction" + 0.005*"baby" + '
  '0.005*"new" + 0.004*"salesforce" + 0.004*"art" + 0.004*"case" + '
  '0.004*"doctor" + 0.003*"come"'),
 (26,
  '0.017*"say" + 0.008*"company" + 0.005*"case" + 0.005*"may" + 0.004*"go" + '
  '0.004*"report" + 0.003*"government" + 0.003*"make" + 0.003*"include" + '
  '0.003*"could"'),
 (8,
  '0.018*"say" + 0.010*"trump" + 0.007*"vote" + 0.006*"support" + '
  '0.006*"people" + 0.005*"conservative" + 0.005*"may" + 0.004*"state" + '
  '0.004*"call" + 0.004*"include"'),
 (18,
  '0.009*"say" + 0.008*"show" + 0.006*"people" + 0.005*"make" + 0.005*"see" + 

In [149]:
# Visualize topics
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.051081  0.006643       1        1  17.482715
8      0.076479 -0.001590       2        1  15.078688
3      0.086773 -0.053560       3        1   7.451463
5      0.055827 -0.025046       4        1   6.958794
12     0.026673  0.064122       5        1   6.467116
10     0.038942 -0.000423       6        1   4.922767
26     0.044469  0.015544       7        1   4.716983
6      0.043443 -0.030563       8        1   3.512779
24     0.035170  0.012411       9        1   3.385217
1      0.028905  0.045388      10        1   3.211503
0      0.021891  0.000657      11        1   2.587369
17     0.030231  0.002306      12        1   2.490953
13     0.027810 -0.014793      13        1   2.453247
23     0.033024 -0.018940      14        1   2.395686
18    -0.008197 -0.039382      15        1   2.319001
9      0.030566 -0.026883      16        1   2.177273
4      0.013298  0.004601      17        1   2.160761
25    -0.001674  0.017282      18        1   1.613029
20     0.018267  0.006140      19        1   1.586031
19    -0.078557  0.184585      20        1   1.347277
7     -0.011202 -0.012584      21        1   1.286618
14    -0.012581 -0.056954      22        1   1.237357
16    -0.060518 -0.008602      23        1   0.929091
21    -0.054663  0.040069      24        1   0.878207
15    -0.073518 -0.003906      25        1   0.728492
22    -0.176550 -0.044480      26        1   0.347045
11    -0.185388 -0.062042      27        1   0.274540, topic_info=        Term          Freq         Total Category  logprob  loglift
243      say  15870.000000  15870.000000  Default  30.0000  30.0000
294    trump   4565.000000   4565.000000  Default  29.0000  29.0000
1430  report   1672.000000   1672.000000  Default  28.0000  28.0000
168     make   3328.000000   3328.000000  Default  27.0000  27.0000
197   people   4221.000000   4221.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
276     take      4.102929   2616.246109  Topic27  -6.2459  -0.5600
168     make      4.101787   3328.906416  Topic27  -6.2462  -0.8011
197   people      4.071553   4221.644482  Topic27  -6.2536  -1.0461
554      run      3.645347   1038.866011  Topic27  -6.3641   0.2454
707    claim      3.567161   1027.881409  Topic27  -6.3858   0.2343

[2249 rows x 6 columns], token_table=       Topic      Freq                          Term
term                                                
11199      8  0.748974                            ab
41645      7  0.876821                         abase
20679      7  0.795662                     abatement
41747     27  0.465869  abbott_northwestern_hospital
29070      1  0.955493                    aboriginal
...      ...       ...                           ...
12989     10  0.849981                         yondr
39081     26  0.435991                     yorkville
28318     15  0.867702                          zaye
17974      2  0.116670                        zombie
17974      6  0.700017                        zombie

[10777 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 4, 6, 13, 11, 27, 7, 25, 2, 1, 18, 14, 24, 19, 10, 5, 26, 21, 20, 8, 15, 17, 22, 16, 23, 12])

In [48]:
# Iterate through corpus to get dominant topic in each document
dominate_topics = list()
for i, row in enumerate(lda_model[corpus]):
  # Select the topic number and percent contribution of the dominant topic
  row = sorted(row, key=lambda x: x[1], reverse=True)[0]
  # Use index to append the text of the document along with the topic number and percent contribution
  dominate_topics.append([' '.join(texts[i]), row[0], row[1]])

# Create a dataframe using with the list from above
topic_sentences_df = pd.DataFrame()
topic_sentences_df[['text', 'topic number', 'percent contribution']] = np.array(dominate_topics, dtype=object)

In [102]:
pd.options.display.max_rows = None
# Group by topic number and show the three documents with the highest contribution for each topic number
topic_sentences_df.groupby('topic number').apply(lambda df: df.sort_values('percent contribution', ascending=False)[:3].drop(columns='topic number', axis=1))

text  percent contribution
topic number                                                                                                                                                                                                                
0            3582  throne season_six include speculation future event time year know annual appearance confuse explain excited know good_chance hype book resurrect murder bolton frey betray group...              0.997616
             3592  question surprisingly fraught answer simple recently compile example man reddit twitter share photo woman find beautiful seemingly amazed enlighten taste woman celebrity believ...              0.996311
             3798  beware late nasty virus sweep particularly elite citadel attack inexplicable fast light palsy stammering overbearing snobbery sometimes redness face include total delusionment\...              0.995701
1            3711  announce policy circle see asylum_seeker transfer first positive news asylum_seeker island circuit breaker receive support country however worth observing could consider good_d...              0.997390
             3646  wake know drive early_hour monday_morning injure several people follow hold meet police_said_statement earlier_report give man age man search carry residential address area mee...              0.997009
             3143  hundred law_enforcement_officer scour orlando flee trigger intense manhunt include search apartment complex place dozen school lockdown officer die search gunman die motorcycle...              0.996282
2            3967  annual migration wetland sure cuckoo go cuckoo tag sensor follow bird know great flier say live chinese capital project track bird migration incredibly perilous bird many peris...              0.997898
             2371  soldier day american_soldier arrive help plan strategic city assault military increasingly find draw back regular combat situation year struggle offensive increase american\npa...              0.997425
             3544  big key sign seem baffle deer stumble confusion howl pain find head hideously disfigure eat alive biologist discover turn even frightening reappearance parasite know new world\...              0.997362
3            3076  pull even ahead series recent unthinkable become least plausible elect must compete political map look forbid republican_primarie prove master nationalize political debate appe...              0.999082
             3452  recent teaching conference session design_thinke education draw capacity crowd teacher demonstrate use concept plan_execute project student ask develop hypothetical city town g...              0.998624
             3269  remember post uniformed guard closely fight state_assembly race hoist sign english_spanish warn noncitizen prohibit voting guard remove state election official threaten\nlegal_...              0.997956
4            3018  ferocious hurricane packing wind prompt hundred guantanamo_bay employee evacuate threaten expect dump torrential rainfall rainfall produce flood say preparation protect life\np...              0.997713
             3391  redd musician hear come town unwilling let presumptive_republican_nominee visit pass quietly create event organize protest title send hell protester visit page arrive arena spe...              0.997168
             2887  make way downtown differ other startling way marcher show type rifle wear openly strap slung_across shoulder back long bastion sentiment kind place openly talk gun carry person...              0.997037
5            2941  taciturn hurry various determination tourist proud ask direction really course follow watch street level surveillance hub side erstwhile employer track go bring back circulatio...              0.997193
             3953  find huge hole final even unsettle latvian dazzle winner everywhere court defeat woman final newly turn trailed set fend break_point avoid falling_behind rally victory trail be...              0.99